In [1]:
import numpy as np
import pandas as pd
from scipy.stats import chi2
import os
import matplotlib.pyplot as plt

In [2]:
# DF_TRAIT, DF_ANNOT, DF_PANNOT
DF_TRAIT = pd.read_csv('/n/groups/price/martin/LDSPEC_data/results/tables/trait_info.tsv', sep='\t')
DF_TRAIT.index = DF_TRAIT['Trait_Identifier']
TRAIT_LIST = list(DF_TRAIT.index)
TRAIT_LIST_INDPT = list(DF_TRAIT.index[DF_TRAIT['Indpt']])
print("n_trait=%d" % len(TRAIT_LIST))
print("n_trait_indpt=%d" % len(TRAIT_LIST_INDPT))

SUMSTATS_FILE='/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/sumstats/@.nomhc.sumstats.gz'

# afreq file 
FREQ_FILE = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/imp_geno_chimp/'\
    'ukb_imp_chr@_v3_chimp.afreq'
dic_freq = {}
for CHR in range(1,23):
    temp_df = pd.read_csv(FREQ_FILE.replace('@', '%d'%CHR), sep='\t')
    temp_dic = {x:y for x,y in zip(temp_df['ID'], temp_df['ALT_FREQS'])}
    dic_freq.update(temp_dic)
    
# df_snp : REF:REF_Chimp, ALT: derived
PVAR_FILE = '/n/groups/price/martin/LDSPEC_data/UKBimp_337K_MAF001_chimp/imp_geno_chimp/'\
    'ukb_imp_chr@_v3_chimp.pvar'
df_snp = []
for CHR in range(1,23):
    df_snp.append(pd.read_csv(PVAR_FILE.replace('@', '%d'%CHR), sep='\t'))
df_snp = pd.concat(df_snp, axis=0)

n_trait=70
n_trait_indpt=29


### Check plink 

In [7]:
BIM_FILE='/n/scratch/users/j/jz286/imp_geno_chimp_bed_MAF01/ukb_imp_chr22_v3_chimp.bim'
df_bim = pd.read_csv(BIM_FILE, sep='\t', header=None)

BIM_FILE_REF='/n/groups/price/martin/gene_dir_effect/imp_geno_chimp_bed_MAF01/ukb_imp_chr22_v3_chimp.bim'
df_bim_ref = pd.read_csv(BIM_FILE_REF, sep='\t', header=None)

for i in range(6):
    print('col', i, (df_bim[i]==df_bim_ref[i]).mean())

col 0 1.0
col 1 1.0
col 2 1.0
col 3 1.0
col 4 1.0
col 5 1.0


### Get SNP list for a gene 

In [8]:
# df_snp from -bfile 
DATA_PATH='/n/scratch/users/j/jz286/imp_geno_chimp_bed_MAF01'
df_list = []
for CHR in range(1,23):
    temp_df = pd.read_csv('%s/ukb_imp_chr%s_v3_chimp.bim'%(DATA_PATH,CHR), sep='\t', header=None)
    df_list.append(temp_df)
df_snp = pd.concat(df_list, axis=0)
df_snp.index = df_snp[1]
print(df_snp)

                  0                1  2         3  4     5
1                                                         
1:612688_TCTC_T   1  1:612688_TCTC_T  0    612688  T  TCTC
rs539032812       1      rs539032812  0    665266  C     T
rs12238997        1       rs12238997  0    693731  G     A
rs189800799       1      rs189800799  0    701835  C     T
rs200531508       1      rs200531508  0    711310  A     G
...              ..              ... ..       ... ..   ...
rs376461333      22      rs376461333  0  51232488  G     A
rs11091014       22       rs11091014  0  51233666  T     C
rs374867791      22      rs374867791  0  51234343  T     G
rs191117135      22      rs191117135  0  51234799  A     G
rs370652263      22      rs370652263  0  51237712  A     G

[9249958 rows x 6 columns]


In [7]:
df_gene = pd.read_csv(
    '/n/groups/price/martin/gene_dir_effect/gene_annotation/glist_ensgid_hg19_v40.txt', sep='\t', header=None,
)
df_gene.loc[df_gene[0].isin(['%d'%x for x in range(1,23)])]
df_gene_id2symbol = pd.read_csv(
    '/n/groups/price/martin/gene_dir_effect/gene_annotation/glist_ensgid_hg19_v40_symbol_gene_names.txt',
    sep='\t', 
)
dic_id2sym = {x:y for x,y in zip(df_gene_id2symbol['ensgid'], df_gene_id2symbol['gene_name'])}

winsize=int(1e5)
OUT_PATH = '/n/groups/price/martin/gene_dir_effect/gene_annotation/gene_level_snp_100kb'
for idx in df_gene.index:
    CHR,START,END,ID = df_gene.loc[idx]
    SYM = dic_id2sym[ID]
    snp_list = df_snp.loc[
        (df_snp[0]==int(CHR)) & (df_snp[3]>START-0.5*winsize) & (df_snp[3]<END+0.5*winsize), 1
    ].tolist()
    print(CHR,START,END,ID,SYM, len(snp_list))
    if len(snp_list)==0:
        continue
    # Write
    with open('%s/%s.%s.%s.snplist' % (OUT_PATH, ID, SYM, CHR), 'w') as f:
        for snp in snp_list:
            f.write('%s\n'%snp)

1 65419 71585 ENSG00000186092 OR4F5 0
1 621096 622034 ENSG00000284662 OR4F16 2
1 859303 879955 ENSG00000187634 SAMD11 437
1 879583 894689 ENSG00000188976 NOC2L 481
1 895964 901099 ENSG00000187961 KLHL17 436
1 901862 911245 ENSG00000187583 PLEKHN1 450
1 910578 917497 ENSG00000187642 PERM1 438
1 934342 935552 ENSG00000188290 HES4 407
1 936518 949920 ENSG00000187608 ISG15 438
1 955500 991498 ENSG00000188157 AGRN 493
1 1006347 1009686 ENSG00000237330 RNF223 382
1 1017198 1051741 ENSG00000131591 C1orf159 539
1 1109260 1133316 ENSG00000162571 TTLL10 660
1 1138888 1141972 ENSG00000186891 TNFRSF18 566
1 1146720 1149533 ENSG00000186827 TNFRSF4 551
1 1152289 1167447 ENSG00000078808 SDF4 613
1 1167617 1170421 ENSG00000176022 B3GALT6 537
1 1177826 1182102 ENSG00000184163 C1QTNF12 503
1 1189289 1209244 ENSG00000160087 UBE2J2 525
1 1215816 1227409 ENSG00000162572 SCNN1D 443
1 1227770 1244989 ENSG00000131584 ACAP3 444
1 1243977 1247057 ENSG00000169972 PUSL1 382
1 1246965 1260067 ENSG00000127054 INTS1

1 13742808 13747803 ENSG00000204478 PRAMEF20 116
1 13801441 13840498 ENSG00000162494 LRRC38 593
1 13909960 13944452 ENSG00000162493 PDPN 608
1 14026683 14151574 ENSG00000116731 PRDM2 844
1 14219287 15444539 ENSG00000189337 KAZN 5721
1 15479028 15546974 ENSG00000171729 TMEM51 1022
1 15563017 15726779 ENSG00000142621 FHAD1 1105
1 15736384 15756835 ENSG00000142634 EFHD2 405
1 15764938 15775737 ENSG00000162438 CTRC 382
1 15783223 15798586 ENSG00000142615 CELA2A 432
1 15792404 15817890 ENSG00000215704 CELA2B 474
1 15817327 15853029 ENSG00000132906 CASP9 502
1 15853308 15918874 ENSG00000116138 DNAJC16 678
1 15898194 15911546 ENSG00000116771 AGMAT 466
1 15943953 15995539 ENSG00000197312 DDI2 587
1 15986208 15988528 ENSG00000215695 RSC1A1 306
1 16010815 16061264 ENSG00000116786 PLEKHM2 454
1 16062753 16067887 ENSG00000162461 SLC25A34 337
1 16068994 16074477 ENSG00000162460 TMEM82 335
1 16083102 16113089 ENSG00000162458 FBLIM1 518
1 16133664 16135843 ENSG00000233954 UQCRHL 405
1 16162590 162669

1 27113545 27127150 ENSG00000060642 PIGV 177
1 27153179 27184095 ENSG00000204160 ZDHHC18 215
1 27189640 27190947 ENSG00000175793 SFN 185
1 27202623 27216774 ENSG00000142751 GPN2 215
1 27216979 27226958 ENSG00000198746 GPATCH3 207
1 27226729 27273362 ENSG00000090273 NUDC 252
1 27237980 27240466 ENSG00000131910 NR0B2 183
1 27276053 27286959 ENSG00000175707 KDF1 178
1 27320183 27327377 ENSG00000253368 TRNP1 193
1 27331511 27339341 ENSG00000158246 TENT5B 204
1 27425300 27493472 ENSG00000090020 SLC9A1 381
1 27561123 27635127 ENSG00000142784 WDTC1 383
1 27648636 27662891 ENSG00000186501 TMEM222 239
1 27668511 27680428 ENSG00000142765 SYTL1 216
1 27680558 27693452 ENSG00000142733 MAP3K6 222
1 27695601 27701315 ENSG00000142748 FCN3 211
1 27705666 27709823 ENSG00000174950 CD164L2 211
1 27719126 27722318 ENSG00000181773 GPR3 187
1 27730734 27816678 ENSG00000158195 WASF2 352
1 27860546 27930942 ENSG00000126705 AHDC1 303
1 27938575 27961696 ENSG00000000938 FGR 233
1 27992575 27998723 ENSG000001267

1 40361098 40367928 ENSG00000116990 MYCL 307
1 40420784 40435640 ENSG00000168389 MFSD2A 326
1 40505905 40538321 ENSG00000131236 CAP1 497
1 40538382 40562932 ENSG00000131238 PPT1 397
1 40627059 40706593 ENSG00000117000 RLF 276
1 40711619 40717356 ENSG00000188800 TMCO2 166
1 40723713 40759852 ENSG00000084073 ZMPSTE24 344
1 40766161 40783485 ENSG00000049089 COL9A2 354
1 40810522 40888998 ENSG00000084070 SMAP2 644
1 40915774 40929390 ENSG00000187801 ZFP69B 422
1 40942962 40962015 ENSG00000187815 ZFP69 485
1 40974413 40981710 ENSG00000164002 EXO5 462
1 40997245 41013839 ENSG00000117010 ZNF684 497
1 41086352 41131354 ENSG00000117016 RIMS3 440
1 41157320 41237275 ENSG00000066136 NFYC 583
1 41249459 41306124 ENSG00000117013 KCNQ4 605
1 41326726 41328035 ENSG00000179862 CITED4 447
1 41444972 41478237 ENSG00000171793 CTPS1 521
1 41481261 41488909 ENSG00000171790 SLFNL1 377
1 41492874 41707978 ENSG00000010803 SCMH1 971
1 41827594 41849262 ENSG00000204060 FOXO6 452
1 41944446 41950354 ENSG00000127

1 54598359 54665746 ENSG00000256407 ENSG00000256407 519
1 54598641 54619443 ENSG00000157211 CDCP2 369
1 54635324 54665709 ENSG00000215883 CYB5RL 435
1 54649714 54691137 ENSG00000116221 MRPL37 451
1 54691106 54879152 ENSG00000157216 SSBP3 998
1 55007930 55104865 ENSG00000162390 ACOT11 846
1 55074854 55089229 ENSG00000162391 FAM151A 540
1 55107427 55175939 ENSG00000184313 MROH7 768
1 55107459 55207981 ENSG00000271723 MROH7-TTC4 869
1 55181534 55208330 ENSG00000243725 TTC4 550
1 55222571 55230196 ENSG00000162396 PARS2 482
1 55245385 55266996 ENSG00000006555 TTC22 571
1 55271736 55307925 ENSG00000162398 LEXM 577
1 55315300 55352868 ENSG00000116133 DHCR24 621
1 55446301 55457961 ENSG00000143001 TMEM61 510
1 55464606 55482845 ENSG00000162399 BSND 535
1 55505120 55530525 ENSG00000169174 PCSK9 450
1 55532032 55681037 ENSG00000162402 USP24 766
1 56639105 56990167 ENSG00000284686 ENSG00000284686 1743
1 56960433 57110974 ENSG00000162407 PLPP3 1019
1 57110987 57181008 ENSG00000162409 PRKAA2 756
1 

...

20 62711435 62731996 ENSG00000125510 OPRL1 385
20 62714729 62715751 ENSG00000171695 LKAAEAR1 306
20 62733747 62873606 ENSG00000196132 MYT1 811
20 62735155 62738918 ENSG00000125522 NPBWR2 349
20 62887048 62919174 ENSG00000203880 PCMTD2 308
21 45666235 45671581 ENSG00000279493 ENSG00000279493 546
21 45642880 45660887 ENSG00000277117 ENSG00000277117 582
21 45553521 45602558 ENSG00000280071 ENSG00000280071 561
21 45548128 45565591 ENSG00000276612 ENSG00000276612 333
21 45527280 45551063 ENSG00000275464 ENSG00000275464 342
21 45516461 45526433 ENSG00000280433 ENSG00000280433 275
21 44985070 44985529 ENSG00000274559 ENSG00000274559 229
21 44834363 44847011 ENSG00000275993 ENSG00000275993 448
21 44473301 44496472 ENSG00000274276 ENSG00000274276 565
21 44513066 44527693 ENSG00000275895 ENSG00000275895 427
21 44527635 44592915 ENSG00000276076 ENSG00000276076 484
21 35747779 35780163 ENSG00000273590 ENSG00000273590 477
21 35772410 35773370 ENSG00000277277 ENSG00000277277 350
21 35786792 35796264

21 43791996 43816200 ENSG00000160183 TMPRSS3 658
21 43823556 43867794 ENSG00000160185 UBASH3A 708
21 43892596 43916356 ENSG00000160188 RSPH1 487
21 43916118 44001550 ENSG00000160190 SLC37A1 813
21 44073731 44195619 ENSG00000160191 PDE9A 1126
21 44263204 44299678 ENSG00000160193 WDR4 825
21 44299754 44333414 ENSG00000160194 NDUFV3 833
21 44336913 44345740 ENSG00000233056 ERVH48-1 638
21 44394620 44454041 ENSG00000160199 PKNOX1 989
21 44473301 44497053 ENSG00000160200 CBS 566
21 44513066 44527680 ENSG00000160201 U2AF1 427
21 44589118 44592915 ENSG00000160202 CRYAA 299
21 44834363 44847011 ENSG00000142178 SIK1 448
21 44949066 45079369 ENSG00000160207 HSF2BP 549
21 44985063 44985529 ENSG00000234289 H2BS1 229
21 45079441 45115960 ENSG00000160208 RRP1B 353
21 45138975 45182188 ENSG00000160209 PDXK 537
21 45192392 45196211 ENSG00000160213 CSTB 494
21 45209394 45225174 ENSG00000160214 RRP1 589
21 45285104 45407473 ENSG00000160216 AGPAT3 756
21 45432190 45526433 ENSG00000160218 TRAPPC10 513
21 

22 24110413 24126503 ENSG00000099953 MMP11 516
22 24129118 24180196 ENSG00000099956 SMARCB1 679
22 24176690 24181315 ENSG00000099958 DERL3 522
22 24198890 24228499 ENSG00000133460 SLC2A11 717
22 24204375 24237410 ENSG00000251357 ENSG00000251357 695
22 24236570 24237414 ENSG00000240972 MIF 452
22 24299601 24303382 ENSG00000133433 GSTT2B 353
22 24309076 24314745 ENSG00000099974 DDTL 300
22 24313554 24322019 ENSG00000099977 DDT 275
22 24313567 24322660 ENSG00000285762 ENSG00000285762 274
22 24407642 24574596 ENSG00000099991 CABIN1 545
22 24577455 24585074 ENSG00000099994 SUSD2 333
22 24615622 24641076 ENSG00000099998 GGT5 450
22 24666785 24813707 ENSG00000100014 SPECC1L 783
22 24666866 24838324 ENSG00000258555 SPECC1L-ADORA2A 872
22 24813847 24838325 ENSG00000128271 ADORA2A 371
22 24890075 24924358 ENSG00000100024 UPB1 391
22 24936391 24951903 ENSG00000138867 GUCD1 334
22 24951926 24978020 ENSG00000100028 SNRPD3 327
22 24951975 25024954 ENSG00000286070 ENSG00000286070 387
22 24981588 2498

22 38507502 38610785 ENSG00000184381 PLA2G6 701
22 38596774 38612514 ENSG00000185022 MAFF 445
22 38615298 38669016 ENSG00000198792 TMEM184B 504
22 38686697 38714089 ENSG00000213923 CSNK1E 329
22 38687924 38794527 ENSG00000283900 TPTEP2-CSNK1E 573
22 38822332 38851205 ENSG00000168135 KCNJ4 363
22 38864083 38879452 ENSG00000100196 KDELR3 291
22 38879443 38903665 ENSG00000100201 DDX17 290
22 38914953 38966209 ENSG00000100206 DMC1 315
22 38974123 39052634 ENSG00000184949 FAM227A 518
22 39052641 39069859 ENSG00000100211 CBY1 401
22 39077962 39081426 ENSG00000100216 TOMM22 355
22 39081548 39097561 ENSG00000100221 JOSD1 380
22 39101747 39134304 ENSG00000100226 GTPBP1 381
22 39129295 39190148 ENSG00000100242 SUN2 419
22 39174513 39190203 ENSG00000100246 DNAL4 315
22 39214457 39240033 ENSG00000221890 NPTXR 401
22 39257427 39268254 ENSG00000183741 CBX6 397
22 39348746 39388783 ENSG00000128383 APOBEC3A 404
22 39378352 39388809 ENSG00000179750 APOBEC3B 328
22 39410262 39416357 ENSG00000244509 APOB

ValueError: invalid literal for int() with base 10: 'X'

In [14]:
# list of chr,gene pairs 
DATA_PATH='/n/groups/price/martin/gene_dir_effect'
str_list = []
for file in os.listdir('%s/gene_annotation/gene_level_snp_100kb'%DATA_PATH):
    CHR = file.split('.')[2]
    str_list.append('%s\t%s\n'%(CHR, file))
with open('%s/gene_annotation/gene_level_snp_100kb.txt'%DATA_PATH, 'w') as f:
    for i_line,line in enumerate(sorted(str_list)):
        f.write('%d\t%s'%(i_line+1,line))

### Unfinished jobs

In [3]:
GENE_FILE='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/gene_annotation/gene_level_snp_100kb.txt'
GENE_FILE_UF='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/gene_annotation/gene_level_snp_100kb_uf.txt'

gene_list = pd.read_csv(GENE_FILE, header=None, sep='\t')[2].tolist()
gene_list_uf = []
for i_gene,gene in enumerate(gene_list):
    for trait in TRAIT_LIST_INDPT:
        file_path='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/res_cojo_100kb/%s/%s.log'%(trait,gene)
        flag_finish = False
        if os.path.exists(file_path):
            with open(file_path, 'r') as f:
                last_line = f.readlines()[-1]
                if last_line.startswith('Overall computational time'):
                    flag_finish = True
        if flag_finish == False:
            CHR = gene.split('.')[2]
            gene_list_uf.append('%s\t%s\n'%(CHR, gene))
            break
            
    if i_gene%100==0:
        print('%s/%s'%(i_gene, len(gene_list)))
    
    
print('n_unfinished=%d/%d' % (len(gene_list_uf), len(gene_list)))

with open(GENE_FILE_UF, 'w') as f:
    for i_line,line in enumerate(sorted(gene_list_uf)):
        f.write('%d\t%s'%(i_line+1,line))

0/18958
100/18958
200/18958
300/18958
400/18958
500/18958
600/18958
700/18958
800/18958
900/18958
1000/18958
1100/18958
1200/18958
1300/18958
1400/18958
1500/18958
1600/18958
1700/18958
1800/18958
1900/18958
2000/18958
2100/18958
2200/18958
2300/18958
2400/18958
2500/18958
2600/18958
2700/18958
2800/18958
2900/18958
3000/18958
3100/18958
3200/18958
3300/18958
3400/18958
3500/18958
3600/18958
3700/18958
3800/18958
3900/18958
4000/18958
4100/18958
4200/18958
4300/18958
4400/18958
4500/18958
4600/18958
4700/18958
4800/18958
4900/18958
5000/18958
5100/18958
5200/18958
5300/18958
5400/18958
5500/18958
5600/18958
5700/18958
5800/18958
5900/18958
6000/18958
6100/18958
6200/18958
6300/18958
6400/18958
6500/18958
6600/18958
6700/18958
6800/18958
6900/18958
7000/18958
7100/18958
7200/18958
7300/18958
7400/18958
7500/18958
7600/18958
7700/18958
7800/18958
7900/18958
8000/18958
8100/18958
8200/18958
8300/18958
8400/18958
8500/18958
8600/18958
8700/18958
8800/18958
8900/18958
9000/18958
9100/18958


In [22]:
# GENE_FILE_UF='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/gene_annotation/gene_level_snp_100kb_uf.txt'
# gene_list_uf = pd.read_csv(GENE_FILE_UF, header=None, sep='\t')[2].tolist()
# for i_gene,gene in enumerate(gene_list_uf):
#     trait_list_uf = []
#     for trait in TRAIT_LIST_INDPT:
#         file_path='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/res_cojo_100kb/%s/%s.log'%(trait,gene)
#         flag_finish = False
#         if os.path.exists(file_path):
#             with open(file_path, 'r') as f:
#                 last_line = f.readlines()[-1]
#                 if last_line.startswith('Overall computational time'):
#                     flag_finish = True
#         if flag_finish == False:
#             trait_list_uf.append(trait)
            
#     if len(trait_list_uf)>0:
#         print(gene, len(trait_list_uf))
        
        
# #     if i_gene>10:
# #         break

ENSG00000271567.PPIAL4E.1.snplist 29
ENSG00000273213.H3-2.1.snplist 29
ENSG00000284662.OR4F16.1.snplist 29
ENSG00000204172.AGAP9.10.snplist 29
ENSG00000204175.GPRIN2.10.snplist 29
ENSG00000183629.GOLGA8G.15.snplist 29
ENSG00000227717.GOLGA6L25P.15.snplist 29
ENSG00000205457.TP53TG3C.16.snplist 29
ENSG00000275034.TP53TG3E.16.snplist 29
ENSG00000274419.TBC1D3D.17.snplist 29
ENSG00000274933.TBC1D3I.17.snplist 29
ENSG00000176695.OR4F17.19.snplist 29
ENSG00000223569.USP17L15.4.snplist 29
ENSG00000248920.USP17L19.4.snplist 29
ENSG00000249104.USP17L17.4.snplist 29
ENSG00000250745.USP17L20.4.snplist 29
ENSG00000250844.USP17L18.4.snplist 29
ENSG00000010704.HFE.6.snplist 29
ENSG00000026950.BTN3A1.6.snplist 29
ENSG00000030110.BAK1.6.snplist 29
ENSG00000066379.POLR1H.6.snplist 29
ENSG00000079689.SCGN.6.snplist 29
ENSG00000079691.CARMIL1.6.snplist 29
ENSG00000096395.MLN.6.snplist 29
ENSG00000096433.ITPR3.6.snplist 29
ENSG00000096654.ZNF184.6.snplist 29
ENSG00000111801.BTN3A3.6.snplist 29
ENSG000001

ENSG00000236104.ZBTB22.6.snplist 29
ENSG00000237441.RGL2.6.snplist 29
ENSG00000237541.HLA-DQA2.6.snplist 29
ENSG00000237649.KIFC1.6.snplist 29
ENSG00000237988.OR2I1P.6.snplist 29
ENSG00000240053.LY6G5B.6.snplist 29
ENSG00000240065.PSMB9.6.snplist 29
ENSG00000241106.HLA-DOB.6.snplist 29
ENSG00000241370.RPP21.6.snplist 29
ENSG00000241404.EGFL8.6.snplist 29
ENSG00000242574.HLA-DMB.6.snplist 29
ENSG00000243649.CFB.6.snplist 29
ENSG00000243729.OR5V1.6.snplist 29
ENSG00000244255.ENSG00000244255.6.snplist 29
ENSG00000244355.LY6G6D.6.snplist 29
ENSG00000244731.C4A.6.snplist 29
ENSG00000248167.TRIM39-RPP21.6.snplist 29
ENSG00000248993.ENSG00000248993.6.snplist 29
ENSG00000250264.ENSG00000250264.6.snplist 29
ENSG00000250641.LY6G6F-LY6G6D.6.snplist 29
ENSG00000254870.ATP6V1G2-DDX39B.6.snplist 29
ENSG00000255152.MSH5-SAPCD1.6.snplist 29
ENSG00000255552.LY6G6E.6.snplist 29
ENSG00000258388.PPT2-EGFL8.6.snplist 29
ENSG00000261272.MUC22.6.snplist 29
ENSG00000263020.ENSG00000263020.6.snplist 29
ENSG000

In [11]:
GENE_FILE='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/gene_annotation/gene_level_snp_100kb.txt'
GENE_FILE_UF='/n/groups/price/martin/LDSPEC_data/res_cojo_rv1/gene_annotation/gene_level_snp_100kb_uf.txt'

gene_list = pd.read_csv(GENE_FILE, header=None, sep='\t')[2].tolist()
gene_list_uf = pd.read_csv(GENE_FILE_UF, header=None, sep='\t')[2].tolist()

In [12]:
[x for x in gene_list if 'PPIAL4E' in x]

['ENSG00000271567.PPIAL4E.1.snplist']

In [20]:
len([x for x in gene_list_uf if (x.startswith('1') is True)])

0

In [19]:
gene_list_uf[-1]

'ENSG00000185775.SPATA31A6.9.snplist'

In [21]:
[x for x in gene_list_uf if 'PPIAL4E' in x]

['ENSG00000271567.PPIAL4E.1.snplist']